In [15]:
# latest update
# April 26, 2017
# Changes done by: Jimmy Vivas
# Added code description

# This code was developed by Jimmy Vivas

# The code will take all CSv files from a specific directory and process them at the same time
# The code will get replace strings in the values for numeric data
#   It will assign an integer value for each distinct string for each column (sensor)
#   The integer value will be reset it for each sensor
#   The code does not evaluare wether or not the sensor should stay. THe analyst will make this decision
#   Using Previse or any other tool
# The code will split the csv file if the result is more than 1MM rows

In [16]:
from os import listdir
import pandas as pd
import math
import numpy as np

In [17]:
# mac Path
#CSVPath1 =  '/Users/jvivas/Documents/XMT Baytwon Sensor Data' \
#            '/Sensor Data/Testing'

CSVPath1 = '/Users/jvivas/Documents/Aspen/PTT GC PoC/Sensor Data/A210 - Quench Oil Tower/To Be Processed'

# Windows Path
# CSVPath1 = 'C:/Users/jvivas/Dropbox/Mtell Customer Projects/XOM BayTown RHC MEA Tower Foaming/Sensor Data/' \
#           'To be Processed'

# CSVPath1 = 'C:/Mtell/Projects/XOM Baytown POC/Sensor Data/ToBeFormatted'

In [18]:
CSVFileList = []
NanPHDTagList = []
CSVFileListAll = listdir(CSVPath1)

m = len(CSVFileListAll)

for i in range(m):
    fileNameStr = CSVFileListAll[i]
    fileStr = fileNameStr.split('.')[0]
    fileExt = fileNameStr.split('.')[1]
    if fileExt == "csv":
        CSVFileList.append(fileNameStr)

n = len(CSVFileList)

# Uncomment following cell for debugging purposes

In [19]:
# pd.set_option('display.max_rows', 500)
# pd.set_option('display.max_columns', 500)
# np.set_printoptions(threshold=np.nan)

In [20]:
i = 0
for i in range(n):
    csvfile = CSVPath1 + "/" + CSVFileList[i]
    # print("Loading Segment: %s" % CSVFileList[i])

    # In[43]:
    print ('Loading ' + CSVFileList[i] + ' file')
    %time df = pd.read_csv(csvfile, index_col=False, sep=',', low_memory=False)
    
    # In[51]:
    # Need to do this to avoid 
    df = df.copy()
    # Change new datetime column to datetime format
    df['Date TIme'] = pd.to_datetime(df['Date TIme'])
    # In[53]:
    # Change datetime column format to look like 01/31/2015 0:00:00
    df['Date TIme'] = df['Date TIme'].dt.strftime('%m/%d/%Y %H:%M:%S')

    # Code to do Transposing
    # Create two dataframes df1 only with tags and descriptions. df2 tag with values
    df1 = df[0:1]  # FIRST ROW
    df2 = df[1:len(df)]  # SECOND TO LAST ROW
    
    # Indexing dataframe df1
    df1 = df1.set_index('Date TIme')

    # Indexing dataframe df2
    df2 = df2.set_index('Date TIme')
    
    print ('Converting df to numeric')
    # Convert columns to numbers those that has string wil be converted to numpy null = NaN
    %time df2_with_nan = df2.apply(lambda x: pd.to_numeric(x, errors='coerce'))

    # Ge the list of sensors with Strings
    df3 = pd.DataFrame(df2_with_nan.isnull().any(axis=0))
    df3 = df3.reset_index()
    SensorsWithStrings = df3[df3[0]==True]['index'].tolist()
    
    # Replace nan with 'null' and create new df
    df2_with_null = df2_with_nan.fillna(value='null')
    
    # Here we filter the columns that are objects and create a dataframe with those columns
    colsObject = df2.select_dtypes(include=['object']).columns
    TotalColumns = df2.columns
    
    # print status
    print (str(len(SensorsWithStrings)) + ' columns contain strings out of ' + str(len(TotalColumns)) + ' columns')
    if (len(SensorsWithStrings)/len(TotalColumns)) > 0.5:
        print ('this process will take some time depending of the size of the file and the PC resources')
    
    # Dropping DATETIME index to merge df1 and df2
    df1 = df1.reset_index(drop=False)
    df2 = df2.reset_index(drop=False)
    
    # Variable initialization
    j = 0
    StringListForAllSensors = []

    # Create dictionary from dataframe columns (sensors) that have strings only
    SensorDictionary = {}.fromkeys(SensorsWithStrings, [])

    # Loop to go thru each column and convert the characters string to numbers    
    
    for j in range(len(SensorsWithStrings)):
        # from IPython.core.debugger import Tracer
        # Tracer()() #this one triggers the debugger
            
        # iterate thru each column in the dataframe
        # for j in range(len(list(SensorDictionary))):
        # update sensor name for each column
        Sensor = list(SensorDictionary)[j]
        print ('Processing Sensor: ' + Sensor)
        # Clear List of String for each Sensor
        SensorStringResult = []
        # Iterate thru each row value
        
        # Get rows that have null in the actual sensor column
        # df2_with_null = df2_with_null[df2_with_null['tagname 1']=='null']['tagname 1']
        result = df2_with_nan[df2_with_nan[Sensor].isnull()][Sensor]
        
        # Convert list of nulls to dataframe and reset the datetime index
        result_df = pd.DataFrame(result)
        result_df = result_df.reset_index()
        
        # debugging
        # print (result_df)
        # debugging

        # Getting list of Strings for Sensor
        SensorStringResult = df2[df2['Date TIme'].isin(result_df.loc[:,"Date TIme"].values.tolist())][Sensor]
        
        # Adding Strings to Sensor in Sensor Dictionary
        SensorDictionary[Sensor] = list(set(SensorStringResult))
        
        # Debugging
        # print (list(set(SensorStringResult)))
        # Debugging
        
        # Adding String for Sensor to General String List
        StringListForAllSensors.extend(SensorStringResult)
        # print (SensorDictionary)
    # print (SensorDictionary)
    # Removing Duplicate Strings
    StringListForAllSensors = list(set(StringListForAllSensors))
    print ('These are all the strings found in the csv file: ' + str(StringListForAllSensors))

Loading A210.csv file
CPU times: user 33 s, sys: 5.53 s, total: 38.5 s
Wall time: 39.2 s
Converting df to numeric
CPU times: user 16 s, sys: 560 ms, total: 16.6 s
Wall time: 17 s
278 columns contain strings out of 278 columns
this process will take some time depending of the size of the file and the PC resources
CPU times: user 3 µs, sys: 0 ns, total: 3 µs
Wall time: 5.01 µs
Processing Sensor: I4-FC2206.PV
['Intf Shut', 'Bad']
Processing Sensor: I4-TI10119.PV
['Intf Shut', 'Bad']
Processing Sensor: I4-TI1619.PV
['Intf Shut']
Processing Sensor: I4-FC1220.PV
['Intf Shut']
Processing Sensor: I4-FC1530.PV
['Intf Shut', 'Bad']
Processing Sensor: I4-TI1118.PV
['Intf Shut']
Processing Sensor: I4-FI2102A.PV
['Intf Shut']
Processing Sensor: I4-FC2601.PV
['Intf Shut', 'Bad']
Processing Sensor: I4-TI2203.PV
['Intf Shut']
Processing Sensor: I4-TC1171.PV
['Intf Shut']
Processing Sensor: I4-FI1240.PV
['Intf Shut', 'Bad']
Processing Sensor: I4-TI1861.PV
['Intf Shut']
Processing Sensor: I4-TI1618.PV
[

In [ ]:
list(SensorDictionary)[1]

In [ ]:
df2_with_nan.dtypes

In [ ]:
df2_with_nan.head()

In [ ]:
df2_with_null = df2_with_nan.fillna(value='null')

In [ ]:
df2_with_null.reset_index()

In [ ]:
result = df2_with_nan[df2_with_nan['tagname 1'].isnull()]['tagname 1']
result_df = pd.DataFrame(result)
result_df = result_df.reset_index()

In [ ]:
result_df

In [ ]:
result_df.loc[:,"Date TIme"].values.tolist()

# how to query dataframe  with multiple values

In [ ]:
SensorStringResult = df2[df2['Date TIme'].isin(result_df.loc[:,"Date TIme"].values.tolist())]['tagname 1']

In [ ]:
SensorStringList = SensorStringResult.tolist()

In [ ]:
SensorStringList

In [ ]:
df3 = pd.DataFrame(df2_with_nan.isnull().any(axis=0))

In [ ]:
df3 = df3.reset_index()

In [ ]:
df3 = pd.DataFrame(df2_with_nan.isnull().any(axis=0))
df3 = df3.reset_index()
SensorsWithStrings = df3[df3[0]==True]['index'].tolist()

In [ ]:
SensorDictionary = {}.fromkeys(SensorsWithStrings, [])

In [ ]:
SensorDictionary

In [ ]:
list(SensorDictionary)[1]

In [ ]:
df2_with_nan[df2_with_nan.isnull().any(axis=1)].columns.tolist()

In [ ]:
result_df['Date TIme'][0]

In [ ]:
df2 = df2.reset_index()

In [ ]:
a = pd.DataFrame(df2[df2['Date TIme']==result_df['Date TIme'][0]]['tagname 1'])

In [ ]:
a['tagname 1'].iloc[0]

In [ ]:
i = 0
for i in range(n):
    csvfile = CSVPath1 + "/" + CSVFileList[i]
    print("Loading Segment: %s" % CSVFileList[i])

    # In[43]:
    print ('Time Loading ' + CSVFileList[i] + ' file')
    %time df = pd.read_csv(csvfile, index_col=False, sep=',', low_memory=False)
    
    # In[51]:
    # Need to do this to avoid 
    df = df.copy()
    # Change new datetime column to datetime format
    df['Date TIme'] = pd.to_datetime(df['Date TIme'])
    # In[53]:
    # Change datetime column format to look like 01/31/2015 0:00:00
    df['Date TIme'] = df['Date TIme'].dt.strftime('%m/%d/%Y %H:%M:%S')

    # Code to do Transposing
    # Create two dataframes df1 only with tags and descriptions. df2 tag with values
    df1 = df[0:1]  # FIRST ROW
    df2 = df[1:len(df)]  # SECOND TO LAST ROW
    
    # Indexing dataframe df1
    df1 = df1.set_index('Date TIme')

    # Indexing dataframe df2
    df2 = df2.set_index('Date TIme')

    # Convert columns to numbers those that has string wont be converted
    df2_with_null = df2.apply(lambda x: pd.to_numeric(x, errors='coerce'))

    # Here we filter the columns that are objects and create a dataframe with those columns
    colsObject = df2.select_dtypes(include=['object']).columns
    TotalColumns = df2.columns
    
    # print status
    print (str(len(colsObject)) + ' columns contain strings out of ' + str(len(TotalColumns)) + ' columns')
    print ('this process will take some time depending of the size of the file and the PC resources')
    
    # Loop to go thru each column and convert the characters string to numbers
    y = 0
    for y in range(len(colsObject)):
        # We don't want to change null to number
        # from IPython.core.debugger import Tracer
        # Tracer()() #this one triggers the debugger
            
        # Create dictionary from dataframe columns (sensors)
        SensorDictionary = {}.fromkeys(list(df1), [])
        %time
        # iterate thru each column in the dataframe
        for j in range(len(list(SensorDictionary))):
            # update sensor name for each column
            Sensor = list(SensorDictionary)[j]
            print ('Processing Sensor: ' + Sensor)
            # Clear List of String for each Sensor
            StringList = []
            # Iterate thru each row value
            for x in range(len(df2)):
                try:
                    # df2.iloc[i].apply(lambda x: pd.to_numeric(x, errors='raise'))
                    df2[Sensor].iloc[x] = float(df2[Sensor].iloc[x])
                except ValueError:
                    # Add String to list
                    StringList.append(str(df2[Sensor].iloc[x]))
                    print ('Can not convert '+ str(df2[Sensor].iloc[x]) + ' to number' + ' in row: ' + str(x) + ' for sensor: ' + Sensor)
                # Add distinct Strings to the sensor Dictionary
                SensorDictionary[Sensor] = list(set(StringList))
            print ('Sensors with String data for file: ' + CSVFileList[i])
            print (SensorDictionary[Sensor])
        print (SensorDictionary)

## Now we have to replace string for specific text

In [ ]:
# if pd.unique(df2[[colsObject[1]]].values.ravel()) != 'null':
# Here we are assign a integer to each unique row value, the integer will increase
# from 0 to x where x is the total unique values
    # df2[[df2.columns.get_loc(colsObject[y])]] = pd.factorize(df2.iloc[:,df2.columns.get_loc(colsObject[y])])[0]

# Replacing known test to null
df2.replace({'test': 'null', 'hello there': 'null', 'wow more text': 'null'}, inplace=True, regex=True)

# Now we convert the data to linear format Previse

In [ ]:
df1 = df1.rename(columns={'Date TIme': 'DATETIME'})
df2 = df2.rename(columns={'Date TIme': 'DATETIME'})

In [ ]:
# Dropping DATETIME index to merge df1 and df2
df1 = df1.reset_index(drop=False)
df2 = df2.reset_index(drop=False)

# Converting Historian files to VTQ format (DATETIME, TAGNAME, DESCRIPTION, VALUE)
mdf = pd.merge(pd.melt(df1, id_vars=['DATETIME'], var_name='TAGNAME',
                       value_name='DESCRIPTION')[['TAGNAME', 'DESCRIPTION']],
               pd.melt(df2, id_vars=['DATETIME'], var_name='TAGNAME',
                       value_name='VALUE'),
               on=['TAGNAME'])

# Sort columns by VTQ format
mdf = mdf[['DATETIME', 'TAGNAME', 'DESCRIPTION', 'VALUE']]

# Here we split the CSV data by file and each file will contain 1 million rows

In [ ]:
# Exporting PHD Tag CSV file
j = 1
rows = 1000000
totalRows = len(mdf)
loops = math.ceil(totalRows/rows) + 1

for j in range(loops): #need to round this
    a = (rows*j) - rows
    if totalRows <= rows:
        b = totalRows
        print('Exporting ' + str(CSVFileList[i].replace('.csv', '')) + ' Historian File')
        print("")
        mdf[a:b].to_csv(CSVPath1 + '/' + str(CSVFileList[i].replace('.csv', '')) + '_Formatted.csv', index=False)
    else:
        if (rows*j) >= totalRows:
            b = totalRows
            print('Exporting ' + str(CSVFileList[i].replace('.csv', '')) + ' chunk' + str(j) + ' Historian File')
            print("")
            mdf[a:b].to_csv(CSVPath1 + '/' + str(CSVFileList[i].replace('.csv', '')) + '_Formatted_chunk' + str(j) + '.csv', index=False)
        else:
            b = (rows*j) - 1
            print('Exporting ' + str(CSVFileList[i].replace('.csv', '')) + ' chunk' + str(j) + ' Historian File')
            print("")
            mdf[a:b].to_csv(CSVPath1 + '/' + str(CSVFileList[i].replace('.csv', '')) + '_Formatted_chunk' + str(j) + '.csv', index=False)